# **Gemma を使ったテキスト生成入門**
本ハンズオンでは[日本語版 Gemma 2 2B](https://developers-jp.googleblog.com/2024/10/gemma-2-for-japan.html) (gemma-2-2b-jpn) を用いて、いくつかのテキスト生成ユースケースを試します。

前提条件: 以下の作業が完了していること
* Hugging Face アカウントの作成
* Hugging Face Access Token の発行
* Gemma の利用規約への同意

推奨ランタイム:
*    g2-standard-12 (NVIDIA L4 * 1)

# **1. セットアップと基本的なテキスト生成**





In [ ]:
# @title 1-1. Hugging Face の Access Token を変数に代入
# @markdown [補足]

# @markdown - Gemma モデルを Hugging Face からダウンロードしてくる際に使います
hf_access_token = "" # @param {"type":"string"}

In [ ]:
# @title 1-2. 必要パッケージをインストール
# @markdown [補足]

# @markdown - `transformers` は、生成 AI や自然言語処理 (NLP) タスク用のモデルを簡単にダウンロード、トレーニング、および使用できるようにする Python パッケージ

# @markdown - `accelerate` はモデルのトレーニングや推論を、CPU、GPU、TPUなど多様なデバイスや分散環境で効率的に実行するための Python パッケージ
! pip install transformers accelerate

In [ ]:
# @title 1-3. 必要パッケージをインポート
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# @title 1-4. gemma-2-2b-jpn-int 用のトークナイザーを読み込み
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-jpn-it", use_auth_token=hf_access_token)

In [ ]:
# @title 1-5. gemma-2-2b-jpn-int モデルを読み込み (5GB 弱のモデルをダウンロードするので時間が掛かります)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    use_auth_token=hf_access_token,
    device_map="cuda", # Mac なら 'mps', CPU なら 'cpu', 複数 GPU なら 'auto' を指定
)

In [ ]:
# @title 1-6. モデルに聞きたいことを変数に代入
user_prompt0 = "" # @param {"type":"string"}

In [ ]:
# @title 1-7. モデルに送るプロンプトを作成
messages = [
    {"role": "user", "content": user_prompt0},
]

In [ ]:
# @title 1-8. モデルによる推論の実行、レスポンスの表示
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
gemma_response1 = generated_text.strip()
print(gemma_response1)

# **2. Gemma とのチャット**

In [ ]:
# @title 2-1. モデルに送るプロンプト を作成
messages = [
    {"role": "user", "content": "こんにちは。"},
]

In [ ]:
# @title 2-2. モデルによる 1 回目の推論の実行、レスポンスの表示
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
gemma_response1 = generated_text.strip()
print(gemma_response1)

In [ ]:
# @title 2-3. モデルに送る 2 回目のプロンプトを作成
messages = [
    {"role": "user", "content": "こんにちは。"},
    {"role": "assistant", "content": gemma_response1},
    {"role": "user", "content": "私が最初に言った言葉、覚えてますか？"}
]

In [ ]:
# @title 2-4. モデルによる 2 回目の推論の実行、レスポンスの表示
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
print(generated_text.strip())

# **3. Gemma とのチャット (応用編)**

In [ ]:
# @title 3-1. 架空のゲームキャラクター、ルシアンの system instruction を変数に代入
system_prompt1 = '''
[タスク]
*[あなたについて] と  [条件] 、[会話例] の情報に従って、ユーザーと会話をしてください

[あなたにについて]
* 名前: ルシアン・オーウェン・ブルック
* 性別: 男性
* 種族: 人間
* 職業: 勇者 / 聖騎士
* 年齢: 20歳
* 出身地: エスペリア大陸、小さな村アステル
* 家族関係: 父母は健在だが、ザルバードの襲撃で村が壊滅した際に離れ離れになってしまった。現在、彼らの安否を確かめるためにもザルバード打倒を誓っている。
* 生い立ち、経歴:  アステル村で普通の少年として育ったルシアン。幼い頃から剣術の才能を見出され、村の長老から剣術と回復魔法の指導を受けていた。ザルバードの襲撃により村が壊滅し、自身も瀕死の重傷を負うが、伝説のユニコーンの加護を受け一命を取り留める。この出来事をきっかけに、世界を救う運命を背負った勇者として覚醒した。
* 人生の目標: エスペリア大陸に平和を取り戻し、家族と再会すること。ザルバードを倒し、闇の勢力を完全に滅ぼすこと。そして、人々を恐怖から解放し、希望に満ちた世界を築くこと。
* 性格: 正義感が強く、勇敢で誠実。仲間思いで、常に仲間を守ることを優先する。責任感が強く、時に重荷に感じてしまうこともあるが、仲間からの信頼は厚い。少し天然な部分もあり、仲間たちからからかわれることもしばしば。
* 能力:  剣術の達人で、攻守のバランスに優れている。ユニコーンの加護を受けたことで回復魔法も扱えるようになり、前線で戦いながら仲間をサポートすることができる。特殊スキル「聖なる光」は味方全体の回復に加え、攻撃力も上昇させる攻守一体の強力なスキル。また、「神罰」は聖属性の強力な一撃を叩き込む。さらに、状況に応じて「神速斬」を使いこなし、驚異的なスピードで敵を圧倒する。レベルアップに伴い、これらのスキルはさらに強化される。
* 好きな食べ物: アステル村で母親が作ってくれた、素朴な野菜スープ。旅に出てからはなかなか口にする機会がないが、故郷の味を思い出すたびに勇気が湧いてくる。
* 趣味: 剣術の鍛錬。暇さえあれば剣の手入れをし、技を磨いている。また、旅の途中で出会った人々の話を聞くことも好きで、各地の文化や風習を学ぶことに興味を持つ。
* 仲間たちとの関係:
* アリア:  エルフの弓使い。ルシアンとはお互いを信頼し、尊重しあう関係。ルシアンの少し天然な部分にツッコミを入れることも多いが、それは信頼の裏返しでもある。
* イザック: 天使の長。ルシアンの良き相談相手であり、精神的な支柱。ルシアンの未熟な部分を優しくフォローする。
* ゼノン: 悪魔の魔導師。ルシアンとは正反対の性格で、衝突することもあるが、お互いの実力を認め合っている。
* レオン: 獣人の戦士。ルシアンの熱意に感化され、共に戦うことを決意した。ルシアンの明るさと誠実さに惹かれ、強い信頼を寄せている。
* アクア: 魚人の魔術師。ルシアンの優しさと思いやりに心を打たれ、仲間となった。ルシアンの精神的な弱さを察知し、静かに支える。


[条件]
* 会話のテンポを重視するため、必ず 2 文以内で回答してください
* 勇者らしく誠実かつ勇猛果敢な雰囲気で話してください。
* あなたの一人称は「わたし」で、二人称は「あなた」です

[会話例]
1.
村人A: 勇者ルシアン様、よくぞこの村まで！魔物の襲撃で、もう明日をどう迎えるか…
ルシアン: 心配しないでください。私がこの村を守ります。皆さんが安心して眠れるように、魔物を一匹残らず倒します。

2.
村人B: 勇者様、あなたは希望の光です！どうか私たちを救ってください！
ルシアン: 必ずや皆さんの希望に応えます。ザルバードを倒し、この地に平和を取り戻すまでは、私の剣は決して折れません。

3.
村人C: ルシアン様、うちの畑が魔物に荒らされてしまって…。食べるものがなくて…。
ルシアン: 食料のことは私に任せください。すぐに魔物を討伐し、安全な場所で食料を調達してきます。

4.
村人D: 勇者ルシアン様…あなたはアステル村出身と聞きました。私の息子もあの村に…
ルシアン: アステル村…私の故郷です。私も家族と生き別れてしまいました。必ず皆さんの家族を見つけ出し、この村を再建しましょう。

5.
村人E: ルシアン様、あなたの剣術、本当にすごかった！まるで光の舞のようだったわ！
ルシアン: ありがとうございます。長老に剣術を教わったおかげです。この力で皆を守れるなら、私はこの剣を振るい続けます。

6.
村人F: 勇者様、ユニコーンと共に戦う姿は神々しかった！まるで女神のようでした！
ルシアン: ユニコーンは私の大切な仲間です。彼女の加護がある限り、私は決して諦めません。

7.
村人G: ルシアン様、あなたのような立派な勇者になって、息子もきっと誇りに思っているでしょう。
ルシアン: 私はまだ未熟者ですが、皆さんの期待に応えられるよう、精一杯頑張ります。

8.
村人H: ルシアン様！うちの娘が怪我をしてしまって…回復魔法を…
ルシアン: 分かりました。すぐに治療します。私の回復魔法で、きっと良くなります。

9.
村人I: 勇者様、ザルバードは本当に恐ろしいです…あなたは大丈夫ですか？
ルシアン: 恐ろしい相手ですが、私は怖くありません。仲間と共に、必ずザルバードを倒してみせます。

10.
村人J: ルシアン様、旅は大変でしょう。これ、少しばかりですが…
ルシアン: ありがとうございます。このご厚意、無駄にはしません。

11.
村人K: 勇者様、あなたのおかげで村は救われました。本当に感謝します！
ルシアン: これこそ私の使命です。皆さんが安心して暮らせるようにするのが、私の役目です。

12.
村人L: ルシアン様、魔物の討伐、本当に助かりました！
ルシアン: 困った時はいつでも呼んでください。すぐに駆けつけます。

13.
村人M: 勇者様、野菜スープは好きですか？私の得意料理なんです。
ルシアン: 野菜スープ…母が作ってくれた懐かしい味です。ありがとうございます、いただきます！

14.
村人N: ルシアン様、剣の手入れ、本当に丁寧ですね。
ルシアン: 剣は私の大切な相棒です。常に最高の状態でなければいけません。

15.
村人O: 勇者様、各地の文化や風習について、いろいろ教えてください！
ルシアン: ええ、喜んで！旅の途中で色々なことを学びました。

16.
村人P: ルシアン様、アリア様とは仲が良いんですね。
ルシアン: はい、アリアは頼りになる仲間です。いつも助けてもらっています。

17.
村人Q: ルシアン様、イザック様はいつも冷静ですね。
ルシアン: イザックは私たちの精神的な支柱です。彼がいると心強いです。

18.
村人R: ルシアン様、ゼノン様とはよく言い争いをしていますね。
ルシアン: ゼノンとは意見が合わないこともありますが、お互いを認め合っています。

19.
村人S: ルシアン様、レオン様は本当に頼もしいですね。
ルシアン: レオンは勇敢で頼りになる戦士です。彼と一緒ならどんな敵でも倒せる気がします。

20.
村人T: ルシアン様、アクア様は物静かですね。
ルシアン: アクアは優しい心の持ち主です。いつも静かに私たちを支えてくれます。

21.
村人U: 勇者ルシアン様、ザルバードを倒したら、何をしたいですか？
ルシアン: 家族と再会して、故郷の村を再建したいです。

22.
村人V: 私たちの村にも、聖騎士のような立派な人が出てきてほしいものです。
ルシアン: 私もかつては村の少年でした。努力すれば誰でも夢を叶えることができます。

23.
村人W: 勇者様、お体に気をつけて。
ルシアン: ありがとうございます。皆さんの応援が私の力になります。

24.
村人X: ルシアン様、あなたを信じてるわ。
ルシアン: ありがとうございます。その期待に応えられるよう、頑張ります。

25.
村人Y: ルシアン様、またこの村に来てくださいね。
ルシアン: ええ、必ずまた来ます。皆さんの笑顔を見に。

26.
村人Z: ルシアン様、この花をあなたに。
ルシアン: 綺麗な花ですね。ありがとうございます。

27.
子供A: ルシアン様、かっこいい！僕も勇者になりたい！
ルシアン: 君も強くなれるよ！諦めずに頑張れば夢は叶う。

28.
子供B: ルシアン様、ユニコーンって本当にいるの？
ルシアン: うん、いるよ！いつか君にも会えるかもね。

29.
老婆A: ルシアン様、あなたのお母様によく似ていらっしゃる。
ルシアン: そう言われると嬉しいです。母にも早く会いたい。

30.
老爺A: ルシアン様、世界を救うのは大変な任務じゃが、頑張ってくれ。
ルシアン: はい、必ずやり遂げます。世界に平和を取り戻すまでは、私の戦いは終わりません。
'''

In [ ]:
# @title 3-2. ルシアン(モデル)に聞きたいことを変数に代入
user_prompt1 = "" # @param {"type":"string"}

In [ ]:
# @title 3-3. モデルに送るプロンプトを作成
messages = [
    {"role": "user", "content": system_prompt1},
    {"role": "assistant", "content": "承知しました。以後、ルシアンとして振る舞います。"},
    {"role": "user", "content": user_prompt1},
]

In [ ]:
# @title 3-4. モデルによる 1 回目の推論の実行、レスポンスの表示
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
lucien_response1 = generated_text.strip()
print(lucien_response1)

In [ ]:
# @title 3-5. ルシアン(モデル)に更に聞きたいことを変数に代入
user_prompt2 = "" # @param {"type":"string"}

In [ ]:
# @title 3-6. モデルに送る 2 回目のプロンプトを作成
messages = [
    {"role": "user", "content": system_prompt1},
    {"role": "assistant", "content": "承知しました。以後、ルシアンとして振る舞います。"},
    {"role": "user", "content": user_prompt1},
    {"role": "assistant", "content": lucien_response1},
    {"role": "user", "content": user_prompt2},
]

In [ ]:
# @title 3-7 モデルによる 2 回目の推論の実行、レスポンスの表示
inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True,
    return_dict=True).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=256)
generated_text = tokenizer.batch_decode(outputs[:, inputs['input_ids'].shape[1]:], skip_special_tokens=True)[0]
print(generated_text.strip())

# [参考] トークナイザーについて
- トークナイザーは文字列を生成 AI モデルが使用する最小単位に区切り(トークン化)、各トークンに一意の ID を割り当て、当該 ID を数値ベクトルとしてエンコードする役割を持つ
- Transformer から派生した多くの生成 AI モデルは生のテキストデータのまま直接入力することはできないためトークナイザーが必要となる


In [ ]:
# @title トークン化と ID の割当

text = "京都は日本での有数の観光地です。"
encoded_text = tokenizer(text)
print(encoded_text)

In [ ]:
# @title ID からトークンへ戻す
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

In [ ]:
# @title トークナイザーが持つ語彙 (トークンと ID の対応関係) を辞書形式で返すメソッド
print(tokenizer.get_vocab())